the purpose of this notebook is to take the big dataframe created in 07.01-baseline_data_merging, and make an aggregate we can use for our baseline "worst case" scenario model. This will then be fed into R, where we will use the MICE package to impute data

changelog:

* 4/16/19: added newagg
* 4/17/19: reformatted the ordering of how code runs, and variable names. added aggregation #3.
* 4/19/19: changed the standardizing so that log(x+1) is now applied prior to standardization. also removed ordinal variables from standardizing algorithm and concat them in later with median 0 and iqr 1 so standardize value is either 0 or 1. values from ordinal are not log transformed.


## last run: 6/9/19: sensitivity analysis 1day timewindow

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from pathlib import Path
import seaborn as sns
import numpy as np
import glob

from sklearn.externals.joblib import Memory
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: The 'cachedir' parameter has been deprecated in version 0.12 and will be removed in version 0.14.
You provided "cachedir='/tmp'", use "location='/tmp'" instead.
  # Remove the CWD from sys.path while we load stuff.


In [2]:
#cohort import

os.chdir('/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling') #use to change working directory
wd= os.getcwd() #'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling'


final_pt_df2 = pd.read_csv(Path(wd + '/data/raw/csv/04042019_final_pt_df2_v.csv') , index_col=0) #only for patients with minimum vitals, 14478 icustay_id
patients= list(final_pt_df2['subject_id'].unique())
hadm_id= list(final_pt_df2['hadm_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())
icustay_id= [int(x) for x in icustay_id]

time: 111 ms


In [3]:
len(patients)

11987

time: 7.11 ms


In [4]:
##24 hr sensitivity
# #importing in all clinical_variable files
# timewindowdays="_24"
# # date= '04042019'
# date='09062019'

# # 78 hr elixhauser-redo 
#importing in all clinical_variable files
timewindowdays="_72"
date='11062019'

os.chdir(r'/Users/geickelb1/Documents/GitHub/mimiciii-antibiotics-modeling/data/processed/')
big_df= pd.read_csv(Path(wd+'/data/processed/merged/{}_longdf_preImp{}.csv'.format(date,timewindowdays),  index_col=0))
big_df= big_df.reset_index(drop=True).iloc[:,1:]


time: 12.8 s


In [5]:
## last minute data cleaning/formatting

# #removing firstpos else neg ssc col
# big_df=big_df.loc[:,list(big_df.columns!="first_pos_else_neg_ssc")]
# #changing pao2/fio2 ratio label
# big_df.loc[big_df['label']=="pao2/fio2",'label']="pao2fio2Ratio"

time: 1.91 ms


In [6]:
#overview of all variables and formats
big_df.groupby('label')['value'].describe().sort_values('unique')

,count,unique,top,freq
label,,,,
leukocyte,14181,2,Neg/Not_tested,12042
vasopressin,17425,2,0.0,13778
rrt,14181,2,0.0,13018
phenylephrine,26338,2,1.0,14377
cancer_elix,14181,2,0.0,12439
o2_flow,14433,2,0.0,13442
norepinephrine,34103,2,1.0,22755
nitrite,14181,2,Neg/Not_tested,13783
dobutamine,15470,2,0.0,13951


time: 1.88 s


## initial data prep

In [7]:
# #converting venttype to category
# big_df.loc[big_df.loc[:,'label']=='vent_recieved','value']= big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'].astype('category')
# #big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'].apply(astype('category'))
# big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'] = pd.Categorical(big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'])

time: 1.79 ms


In [8]:
#converting categories to integers
mapper={'Mech':2 , 'Oxygen': 1, 'None': 0}
big_df.loc[big_df.loc[:,'label']=='vent_recieved','value']=big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'].replace(mapper)

#gender_index=pd.factorize(big_df.loc[big_df.loc[:,'label']=='gender','value'])[1]
mapper={'F':0 , 'M': 1}
big_df.loc[big_df.loc[:,'label']=='gender','value']=big_df.loc[big_df.loc[:,'label']=='gender','value'].replace(mapper)

#leukocyte_index=pd.factorize(big_df.loc[big_df.loc[:,'label']=='leukocyte','value'])[1]
mapper={'Neg/Not_tested':0 , 'pos': 1}
big_df.loc[big_df.loc[:,'label']=='leukocyte','value']=big_df.loc[big_df.loc[:,'label']=='leukocyte','value'].replace(mapper)
#     pd.factorize(big_df.loc[big_df.loc[:,'label']=='leukocyte','value'])[0] 

#nitrite_index=pd.factorize(big_df.loc[big_df.loc[:,'label']=='nitrite','value'])[1]
mapper={'Neg/Not_tested':0 , 'pos': 1}
big_df.loc[big_df.loc[:,'label']=='nitrite','value']=big_df.loc[big_df.loc[:,'label']=='nitrite','value'].replace(mapper)
#     pd.factorize(big_df.loc[big_df.loc[:,'label']=='nitrite','value'])[0] 


time: 4.22 s


In [9]:
big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'].value_counts()

2    7139
1    6175
0     746
Name: value, dtype: int64

time: 300 ms


In [10]:
big_df.loc[big_df.loc[:,'label']=='vent_recieved','value'].value_counts()

2    7139
1    6175
0     746
Name: value, dtype: int64

time: 297 ms


#exploring alternative 

# changing pao2/fio2 ratio to a category
A PaO2/FiO2 ratio less than or equal to 200 is necessary for the diagnosis of acute respiratory distress syndrome by the AECC criteria.[6] 
The more recent Berlin criteria defines mild ARDS at a ratio of <300.

A PaO2/FiO2 ratio less than or equal to 250 is one of the minor criteria for severe community acquired pneumonia (i.e., possible indication for inpatient treatment).

A PaO2/FiO2 ratio less than or equal to 333 is one of the variables in the SMART-COP risk score for intensive respiratory or vasopressor support in community-acquired pneumonia.



In [11]:
pd.to_numeric(big_df[big_df['label']=="pao2fio2ratio"]['value'])
labels= ['0-200', '201-333', "334-475","476+"]
bins = pd.IntervalIndex.from_tuples([(0, 200), (200, 333), (333, 475),(475,3000)])

big_df.loc[big_df['label']=="pao2fio2ratio",'value']=pd.cut(pd.to_numeric(big_df.loc[big_df['label']=="pao2fio2ratio",'value']), bins,right=False, labels=labels)
big_df.loc[big_df['label']=="pao2fio2ratio",'value'].value_counts()


(475, 3000]    9634
(0, 200]       1878
(200, 333]     1417
(333, 475]     1252
Name: value, dtype: int64

time: 1.8 s


## calc median/iqr for standardization
take all non-categorical variables for HEALTHY PATIENTS and calculate the median and IQR for them. then will use this to make z scores via: x-median/IQR

### june 13: changed sparse to catgorical

In [12]:
continuous=['daily_sofa',
            'lactate',
            'mingcs',
            'diasbp',
            'heartrate',
            'meanartpress',
            'resprate',
            'sysbp',
            'temperature',
            'hemoglobin',
            'platelet',
            'wbc',
            'calcium',
            'glucose',
            'ph',
            'bicarbonate',
            'bun',
            'chloride',
            'creatinine',
            'inr',
            'potassium',
            'ptt',
            'sodium',
            'bilirubin',
            'spo2',
            'sum_elix']


onetime=['first_admit_age','height','weight']

vaso_active=['phenylephrine',
            'norepinephrine',
            'vasopressin',
            'dobutamine',
            'dopamine',
            'epinephrine'] 

ordinal=[
            'leukocyte',
            'nitrite',
            'vent_recieved',
            'o2_flow',
            'rrt',
            'pao2fio2ratio',
            'cancer_elix',
            "all_vasoactives",
            'bands', #added 6/13/19
            'pco2' #added 6/13/19
]

categorical=[
            "ethnicity",
            'gender'
]


time: 23.8 ms


In [13]:
final_bin=final_pt_df2[['icustay_id',"final_bin"]].copy()
#list of healthy pt
healthy_pt=list(final_bin[final_bin['final_bin']=="C_neg/A_partial"]['icustay_id'])
del(final_bin)

time: 11.8 ms


In [14]:
#splitting big_df, making a copy and restricting it to all values that will be usd in standardizing
healthy_df=big_df.copy()
healthy_df=healthy_df[healthy_df['label'].isin(continuous+onetime)]

healthy_df['value']= pd.to_numeric(healthy_df['value']) #converting to numeric 

#filter to only healthy patients filter
healthy_df=healthy_df[healthy_df['icustay_id'].isin(healthy_pt)].copy() #only numerical values for cneg/ab partial pt

time: 5.74 s


In [15]:
#finding the last 24 hours of each healthy patient.
healthy_pt_end=pd.DataFrame(healthy_df.groupby("icustay_id")['delta'].max())

healthy_pt_end["start"]=healthy_pt_end['delta']- pd.to_timedelta("1 day 00:00:00")
healthy_pt_end=healthy_pt_end.rename(columns={"delta":"end"}).reset_index()

healthy_df= pd.merge(healthy_df,healthy_pt_end, left_on="icustay_id", right_on="icustay_id", how="left" ) #now have the last 24 hours annotated for each patient as start and end.
healthy_df.head()

,icustay_id,uom,label,value,t_0,delta,source,subject_id,end,start
0,200012.0,daily_sofa_score,daily_sofa,1.00,2153-12-23,0 days 00:00:00.000000000,sofa,28448,0 days 15:00:00.000000000,-1 days +15:00:00
1,200012.0,years,first_admit_age,32.99,2153-12-23,0 days 00:00:00.000000000,pt_info,28448,0 days 15:00:00.000000000,-1 days +15:00:00
2,200012.0,elixhauser_comorb_sum,sum_elix,0.00,2153-12-23,0 days 00:00:00.000000000,sum_elix,28448,0 days 15:00:00.000000000,-1 days +15:00:00
3,200012.0,kg,weight,51.20,2153-12-23,0 days 00:00:00.000000000,weight,28448,0 days 15:00:00.000000000,-1 days +15:00:00
4,200012.0,mmol/L,lactate,1.30,2153-12-23,0 days 02:36:00.000000000,labs,28448,0 days 15:00:00.000000000,-1 days +15:00:00


time: 2.26 s


In [16]:
#calculating medians and iqr for each label based on healthy patient's last 24 hours in icu
median_label=pd.DataFrame((healthy_df.groupby("label")['value'].median())).reset_index()
median_label=median_label.rename(columns={'value':"median"})
iqr_label=pd.DataFrame((healthy_df.groupby("label")['value'].quantile(0.75)-healthy_df.groupby("label")['value'].quantile(0.25))).reset_index()
iqr_label=iqr_label.rename(columns={'value':"iqr"})
median_label=pd.merge(median_label,iqr_label)  #final median df
del(iqr_label)
        
#saving the median label for standardization, will be used later on in teh notebook
# pd.DataFrame(median_label).to_csv(Path(
#     wd+'/data/processed/merged/%s_healthy_medians.csv' %(date)),index=False)

time: 1.68 s


In [17]:
median_label

,label,median,iqr
0,bicarbonate,24.00,5.000000
1,bilirubin,0.80,1.600000
2,bun,20.00,21.000000
3,calcium,1.14,0.110000
4,chloride,105.00,8.000000
5,creatinine,1.00,0.900000
6,daily_sofa,3.00,3.000000
7,diasbp,59.00,18.000000
8,first_admit_age,65.15,26.085000
9,glucose,127.00,56.000000


time: 18.4 ms


# Standardizing data

### dataformatting: convert all dtypes to a numeric type that pereserves nan. 

In [18]:
#splitting categorical, ordinal and continuous
big_categorical= big_df.loc[big_df.loc[:,'label'].isin(categorical),:].copy() 

#continuous and ordinal variables
big_noCat1= big_df.loc[big_df.loc[:,'label'].isin(continuous),:].copy() 
big_noCat1['value']= big_noCat1['value'].apply(pd.to_numeric, args=('coerce',)) #instead of convert to float, may preserve nan's better. 

time: 2min 56s


### adding a standardized value (x-median)/iqr  where median is of the last 24 hours in time window for culture neg/ ab partial patients 

In [19]:
big_noCat=pd.merge(big_noCat1, median_label, how="left") #

#loging values
big_noCat['median']= np.log(big_noCat['median']+1.0)
big_noCat['iqr']= np.log(big_noCat['iqr']+1.0)
big_noCat['raw_value']=big_noCat['value']
big_noCat['value']=np.log(big_noCat['value']+1.0)
##

time: 2.42 s


In [20]:
big_noCat['standardize']=((big_noCat['value']-big_noCat['median'])/big_noCat['iqr']).fillna(0) #standardize is log standardized

#making an ordinal df to concat on
ord_df=big_df.loc[big_df.loc[:,'label'].isin(ordinal+vaso_active),:].copy()
ord_df['raw_value']=ord_df['value']
ord_df['standardize']=ord_df['value']
ord_df['median']=None
ord_df['iqr']=None

big_noCat=pd.concat([big_noCat,ord_df], sort=False)
del(ord_df)

#making all ordinal values in standardize equal to unstandardized
big_noCat.loc[big_noCat['label'].isin(ordinal+vaso_active),'standardize']=big_noCat.loc[big_noCat['label'].isin(ordinal+vaso_active),'value']  #do i need to add sparse?
big_noCat.head()

,icustay_id,uom,label,value,t_0,delta,source,subject_id,median,iqr,raw_value,standardize
0,200012.0,daily_sofa_score,daily_sofa,0.693147,2153-12-23,0 days 00:00:00.000000000,sofa,28448,1.38629,1.38629,1,-0.5
1,200012.0,elixhauser_comorb_sum,sum_elix,0,2153-12-23,0 days 00:00:00.000000000,sum_elix,28448,0,1.38629,0,0
2,200012.0,mmol/L,lactate,0.832909,2153-12-23,0 days 02:36:00.000000000,labs,28448,0.993252,0.875469,1.3,-0.183151
3,200012.0,gcs_score,mingcs,2.77259,2153-12-23,0 days 05:12:00.000000000,gcs,28448,2.70805,1.09861,15,0.0587455
4,200012.0,mmHg,diasbp,4.38203,2153-12-23,0 days 06:16:00.000000000,vitals,28448,4.09434,2.94444,79,0.0977035


time: 9.88 s


In [21]:
big_noCat['standardize']= big_noCat['standardize'].apply(pd.to_numeric, args=('coerce',)) #errors occuring downstream due to not having numeric, trying this 4/18/19

time: 32.8 s


# aggregation1:
### clincally guided min/max

In [22]:
hi_value= ['bands',
'bilirubin',
'bun',
'chloride',
'creatinine',
'daily_sofa',
'glucose',
'heartrate',
'inr',
'lactate',
'pco2',
'potassium',
'ptt',
'resprate',
'temperature',
'weight', 'rrt', 
'phenylephrine', 'norepinephrine', 'vasopressin', 'dobutamine', 'dopamine', 'epinephrine',  #added this and removed individual vasoactive 5/3/19
'first_admit_age','leukocyte','nitrite','vent_recieved','o2_flow', 
'any_vasoactive', #added this and removed individual vasoactive 5/3/19
'sum_elix', #added 6/4/19
'cancer_elix'  ]#added 6/4/19

low_value=['bicarbonate',
'diasbp',
'hemoglobin',
'meanartpress',
'mingcs',
'ph',
'platelet',
'spo2',
'sysbp']

both_value=['calcium',
'sodium',
'wbc']

#important_ordinal=["any_vasoactive"]
important_onetime=['first_admit_age','weight','pao2fio2ratio']


time: 14.6 ms


In [23]:
##combining vasoactives
vaso_active_df=big_noCat[big_noCat['label'].isin(vaso_active)].groupby('icustay_id')['value'].max().reset_index()
vaso_active_df['uom']="y/n"
vaso_active_df['label']="any_vasoactive"
vaso_active_df['delta']=pd.to_timedelta("0days")
vaso_active_df['source']="all_vasoactives"

vaso_active_df=pd.merge(vaso_active_df, final_pt_df2[["icustay_id",'subject_id',"t_0"]], how="left", left_on="icustay_id", right_on="icustay_id")
vaso_active_df=pd.merge(vaso_active_df, median_label, how="left").fillna(0)
vaso_active_df['standardize']=vaso_active_df['value']
vaso_active_df['raw_value']=vaso_active_df['value']
vaso_active_df.head()

# # #grabing the rest of the variables not suitable for range or mean/std
big_noCat=pd.concat([big_noCat, vaso_active_df], sort=False)

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: Passing integers to fillna is deprecated, will raise a TypeError in a future version.  To retain the old behavior, pass pd.Timedelta(seconds=n) instead.
  if __name__ == '__main__':


time: 5.13 s


### running the min/max aggregations.



In [24]:
##choose here if wanna use standardization or raw values.
values="standardize"

time: 641 µs


In [25]:
big_noCat.loc[:,'label'].unique()

array(['daily_sofa', 'sum_elix', 'lactate', 'mingcs', 'diasbp',
       'heartrate', 'meanartpress', 'resprate', 'spo2', 'sysbp',
       'temperature', 'hemoglobin', 'platelet', 'wbc', 'calcium',
       'glucose', 'ph', 'bicarbonate', 'bun', 'chloride', 'creatinine',
       'inr', 'potassium', 'ptt', 'sodium', 'bilirubin', 'cancer_elix',
       'dobutamine', 'dopamine', 'epinephrine', 'norepinephrine',
       'o2_flow', 'pao2fio2ratio', 'pco2', 'phenylephrine', 'rrt',
       'vasopressin', 'vent_recieved', 'bands', 'leukocyte', 'nitrite',
       'any_vasoactive'], dtype=object)

time: 312 ms


In [26]:
big_max= big_noCat.loc[big_noCat.loc[:,'label'].isin(hi_value),:]
table = pd.pivot_table(big_max, values=values, columns='label', index=['icustay_id'],aggfunc=max, dropna=False)

time: 1min 36s


In [27]:
big_max.loc[:,'label'].unique()

array(['daily_sofa', 'sum_elix', 'lactate', 'heartrate', 'resprate',
       'temperature', 'glucose', 'bun', 'chloride', 'creatinine', 'inr',
       'potassium', 'ptt', 'bilirubin', 'cancer_elix', 'dobutamine',
       'dopamine', 'epinephrine', 'norepinephrine', 'o2_flow', 'pco2',
       'phenylephrine', 'rrt', 'vasopressin', 'vent_recieved', 'bands',
       'leukocyte', 'nitrite', 'any_vasoactive'], dtype=object)

time: 131 ms


In [28]:
big_max[big_max['label']=='leukocyte']['value'].describe() #good the categorical stuff worked

count     14181
unique        2
top           0
freq      12042
Name: value, dtype: int64

time: 153 ms


In [29]:
big_min= big_noCat.loc[big_noCat.loc[:,'label'].isin(low_value),:]
table2 = pd.pivot_table(big_min, values=values, columns='label', index=['icustay_id'],aggfunc=min, dropna=False)

time: 28.6 s


# qc why pao2fiotRatio isn't working --Fixed

In [30]:
big_noCat['label'].unique()

array(['daily_sofa', 'sum_elix', 'lactate', 'mingcs', 'diasbp',
       'heartrate', 'meanartpress', 'resprate', 'spo2', 'sysbp',
       'temperature', 'hemoglobin', 'platelet', 'wbc', 'calcium',
       'glucose', 'ph', 'bicarbonate', 'bun', 'chloride', 'creatinine',
       'inr', 'potassium', 'ptt', 'sodium', 'bilirubin', 'cancer_elix',
       'dobutamine', 'dopamine', 'epinephrine', 'norepinephrine',
       'o2_flow', 'pao2fio2ratio', 'pco2', 'phenylephrine', 'rrt',
       'vasopressin', 'vent_recieved', 'bands', 'leukocyte', 'nitrite',
       'any_vasoactive'], dtype=object)

time: 306 ms


In [31]:
#table3 is aggregating for min and max
big_both= big_noCat.loc[big_noCat.loc[:,'label'].isin(both_value),:]
table3 = pd.pivot_table(big_both, values=values, columns='label', index=['icustay_id'],aggfunc=[max,min], dropna=False)

time: 16.9 s


In [32]:
print(len(table),len(table2),len(table3)) #14668 14668 14618

14181 14181 14131
time: 1.7 ms


In [33]:
#first left join all different continuous aggregations together. 
worst_df=pd.merge(table.reset_index(), table2.reset_index(), how='left')
worst_df=pd.merge(worst_df, table3.reset_index(), left_on='icustay_id', right_on='icustay_id',how='left')
worst_df.head()

/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/reshape/merge.py:522: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
/Users/geickelb1/anaconda/envs/rpy-env/lib/python3.6/site-packages/pandas/core/generic.py:3812: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


,icustay_id,any_vasoactive,bands,bilirubin,bun,cancer_elix,chloride,creatinine,daily_sofa,dobutamine,...,ph,platelet,spo2,sysbp,"(max, calcium)","(max, sodium)","(max, wbc)","(min, calcium)","(min, sodium)","(min, wbc)"
0,200012.0,0.0,NaN,NaN,NaN,0,NaN,NaN,-0.5,0,...,NaN,-0.076758,-0.012811,-0.062544,NaN,NaN,-0.306266,NaN,NaN,-0.306266
1,200014.0,0.0,NaN,NaN,0.056406,0,0.0370891,-0.16415,-0.207519,0,...,-0.027694,-0.103698,-0.012811,-0.150392,0.437620,0.015722,0.176440,-0.044882,-0.004001,0.040547
2,200019.0,0.0,NaN,NaN,0.056406,0,0.0168582,0,0.403677,0,...,-0.013839,-0.066001,-0.274526,-0.154522,NaN,0.003972,0.149235,NaN,0.003972,0.124170
3,200030.0,1.0,NaN,2.2533,0.104422,0,0.0250588,0,0.292481,1,...,-0.013839,-0.035454,-0.032538,-0.028245,0.815086,0.007917,0.163020,0.044672,-0.004001,0.113037
4,200033.0,1.0,NaN,NaN,-0.0683618,1,0.0127018,-0.253202,-0.5,0,...,0.110121,-0.109168,-0.006373,-0.050656,-0.365081,0.023421,-0.109531,-0.505654,-0.024447,-0.205394


time: 181 ms


In [34]:
### formatting categorical to wide format to match the tables/worst_df
big_categorical= big_categorical.pivot(
    index='icustay_id',
    values='value',
    columns='label').reset_index() #need to convert to wide format. should be one row per patient per time. 

time: 36.3 ms


In [35]:
def merge_cat_agg(num_df, cat_df, twoclass=False):
    """
    merges the categorical and aggregated dataframes together. if twoclass=True (default=False), returns just the truepos trueneg twoclass patients.
    """
    
    worst_df=pd.merge(num_df, cat_df, left_on='icustay_id', right_on='icustay_id',how='left')
#     worst_df['leukocyte']=worst_df['leukocyte'].astype("category")
#     worst_df['nitrite']=worst_df['nitrite'].astype("category")
#     worst_df['vent_recieved']=worst_df['vent_recieved'].astype("category")
    worst_df['ethnicity']=worst_df['ethnicity'].astype("category")
    worst_df['gender']=worst_df['gender'].astype("category")
    
    baselinePts= final_pt_df2.loc[final_pt_df2.loc[:,'final_bin'].isin(["C_pos/A_full","C_neg/A_partial"]),"icustay_id"]
    
    if twoclass==True:
        worst_df= worst_df.loc[worst_df.loc[:,'icustay_id'].isin(baselinePts),:]
    return(worst_df)


time: 21.7 ms


In [36]:
worst_df=merge_cat_agg(worst_df, big_categorical, twoclass=False) #using max/min aggregates 


time: 50.3 ms


In [37]:
#adding important one_times to final aggregated
agg_remaining= big_df.loc[big_df.loc[:,'label'].isin(important_onetime),:]
agg_table2 = pd.pivot_table(agg_remaining, values='value', columns='label', index=['icustay_id'],aggfunc=[max], dropna=False) 
agg_table2.columns = agg_table2.columns.get_level_values(1)
agg_table2=agg_table2.reset_index()
agg_table2.head()#.rename(columns={})

worst_df=pd.merge(worst_df, agg_table2, how='left')

time: 9.65 s


In [38]:
worst_df['pao2fio2ratio'].describe()

count           14181
unique              4
top       (475, 3000]
freq             9634
Name: pao2fio2ratio, dtype: object

time: 24.6 ms


In [39]:
list(worst_df)

['icustay_id',
 'any_vasoactive',
 'bands',
 'bilirubin',
 'bun',
 'cancer_elix',
 'chloride',
 'creatinine',
 'daily_sofa',
 'dobutamine',
 'dopamine',
 'epinephrine',
 'glucose',
 'heartrate',
 'inr',
 'lactate',
 'leukocyte',
 'nitrite',
 'norepinephrine',
 'o2_flow',
 'pco2',
 'phenylephrine',
 'potassium',
 'ptt',
 'resprate',
 'rrt',
 'sum_elix',
 'temperature',
 'vasopressin',
 'vent_recieved',
 'bicarbonate',
 'diasbp',
 'hemoglobin',
 'meanartpress',
 'mingcs',
 'ph',
 'platelet',
 'spo2',
 'sysbp',
 ('max', 'calcium'),
 ('max', 'sodium'),
 ('max', 'wbc'),
 ('min', 'calcium'),
 ('min', 'sodium'),
 ('min', 'wbc'),
 'ethnicity',
 'gender',
 'first_admit_age',
 'pao2fio2ratio',
 'weight']

time: 4.69 ms


In [40]:
worst_df['vasopressin'].value_counts()

0.0    13778
1.0      403
Name: vasopressin, dtype: int64

time: 8.69 ms


In [41]:
date

'11062019'

time: 3.86 ms


In [43]:
##saving
pd.DataFrame(worst_df).to_csv(Path(
    wd+'/data/processed/merged/{}_worstdf_preImp{}.csv'.format(date,timewindowdays)),index=False)

time: 936 ms


In [ ]:
print(wd+'/data/processed/merged/{}_worstdf_preImp{}.csv'.format(date,timewindowdays))

# aggregation 2:
## 4-11-19: range, mean and stdev aggregations (newagg)

np.ptp = difference btween min and max


In [ ]:
##choose here if wanna use standardization or raw values.
values="standardize"

In [ ]:
big_range= big_noCat.loc[big_noCat.loc[:,'label'].isin(continuous),:]
table4 = pd.pivot_table(big_range, values=values, columns='label', index=['icustay_id'],aggfunc=[np.ptp, np.median, np.std], dropna=[False,False,False]) 
table4.columns=['_'.join(col).strip() for col in table4.columns.values]
table4=table4.reset_index()

In [ ]:
#grabing the rest of the variables not suitable for range or mean/std
remaining= big_noCat.loc[big_noCat.loc[:,'label'].isin(onetime+ordinal),:]
table5 = pd.pivot_table(remaining, values='value', columns='label', index=['icustay_id'],aggfunc=[max], dropna=False) 

In [ ]:
#table5.columns
table5.columns = table5.columns.get_level_values(1)
table5=table5.reset_index()

In [ ]:
table4.head()#.rename(columns={})

In [ ]:
## merging aggregations together
#first left join all different newagg together. 
table4=pd.merge(table4, table5, how='left')

In [ ]:
list(table4)[1:20]

In [ ]:
### formatting categorical to wide format to match the aggregated numerical
big_categorical= big_df.loc[big_df.loc[:,'label'].isin(categorical),:].copy()
big_categorical= big_categorical.pivot(
    index='icustay_id',
    values='value',
    columns='label').reset_index() #need to convert to wide format. should be one row per patient per time. 

### merging categorical with aggregated

In [ ]:
#using fxn written for worst_df to make newagg_df
newagg_df=merge_cat_agg(table4, big_categorical, twoclass=False) #using range/mean/stdev aggregates 


In [ ]:
newagg_df.head()

In [ ]:
list(newagg_df)

In [ ]:
pd.DataFrame(newagg_df).to_csv(Path(
    wd+'/data/processed/merged/%s_newaggdf_std.csv' %(date)),index=False)

In [ ]:
newagg_df['leukocyte']

In [ ]:
wd+'/data/processed/merged/%s_newaggdf_std.csv'

# aggregation 3:
# grabbing min/max/median/sd for only values found to be important from combining variable importance for random forest, boosted trees, and logistic regression on the baseline clinical max/min data.


In [ ]:
# important_variables=["temperature","sysbp","daily_sofa","meanartpress","bun","platelet","ptt","heartrate","pao2fio2ratio","creatinine","first_admit_age"]
important_continuous=["temperature","sysbp","daily_sofa","meanartpress","bun","platelet","ptt","heartrate","creatinine"]
important_ordinal=["any_vasoactive", 'vent_recieved', "leukocyte","pao2fio2ratio"]
important_onetime=['first_admit_age','weight']



In [ ]:
# #continuous and ordinal variables
# big_noCat= big_df.loc[big_df.loc[:,'label'].isin(continuous+ordinal),:].copy()
# big_noCat['value']= big_noCat['value'].apply(pd.to_numeric, args=('coerce',)) #instead of convert to float, may preserve nan's better. 

In [ ]:
big_noCat[big_noCat['label'].isin(vaso_active)].groupby('icustay_id')['value'].max().reset_index()

In [ ]:
##combining vasoactives
vaso_active_df=big_noCat[big_noCat['label'].isin(vaso_active)].groupby('icustay_id')['value'].max().reset_index()
vaso_active_df['uom']="y/n"
vaso_active_df['label']="any_vasoactive"
vaso_active_df['delta']=pd.to_timedelta("0days")
vaso_active_df['source']="all_vasoactives"

vaso_active_df=pd.merge(vaso_active_df, final_pt_df2[["icustay_id",'subject_id',"t_0"]], how="left", left_on="icustay_id", right_on="icustay_id")
vaso_active_df=pd.merge(vaso_active_df, median_label, how="left").fillna(0)
vaso_active_df['standardize']=vaso_active_df['value']
vaso_active_df['raw_value']=vaso_active_df['value']
vaso_active_df.head()

In [ ]:
vaso_active_df.iloc[3,1]

In [ ]:
big_noCat.head()

In [ ]:
big_noCat['label'].unique()
#big_noCat=pd.concat([big_noCat, vaso_active_df], sort=False)

In [ ]:
big_noCat[big_noCat['label']=="any_vasoactive"].head() #so this works

In [ ]:
##choose here if wanna use standardization or raw values.
values="standardize"
#big_noCat[values]= big_noCat[values].apply(pd.to_numeric, args=('coerce',)) #instead of convert to float, may preserve nan's better.

In [ ]:
# big_df2=pd.concat([big_df, vaso_active_df], sort=False)

# #splitting categorical, ordinal and continuous
# big_categorical= big_df.loc[big_df.loc[:,'label'].isin(important_ordinal+important_onetime),:].copy()

# #continuous and ordinal variables
# df_continuous= big_df.loc[big_df.loc[:,'label'].isin(important_continuous+ordinal),:].copy()
# big_noCat['value']= big_noCat['value'].apply(pd.to_numeric, args=('coerce',)) #instead of convert to float, may preserve nan's better. 

In [ ]:
df_continuous= big_noCat.loc[big_noCat.loc[:,'label'].isin(important_continuous),:]
agg_table1 = pd.pivot_table(df_continuous, values=values, columns='label', index=['icustay_id'],aggfunc=[np.max, np.min, np.median, np.std], dropna=[False,False,False, False]) 
agg_table1.columns=['_'.join(col).strip() for col in agg_table1.columns.values]
agg_table1=agg_table1.reset_index()

In [ ]:
#code above is not 

In [ ]:
#grabing the rest of the variables not suitable for range or mean/std
big_df2=pd.concat([big_df, vaso_active_df], sort=False)

agg_remaining= big_df2.loc[big_df2.loc[:,'label'].isin(important_onetime+important_ordinal),:]
agg_table2 = pd.pivot_table(agg_remaining, values='value', columns='label', index=['icustay_id'],aggfunc=[max], dropna=False) 
agg_table2.columns = agg_table2.columns.get_level_values(1)
agg_table2=agg_table2.reset_index()
agg_table2.head()#.rename(columns={})

agg_cont_cat=pd.merge(agg_table1, agg_table2, how='left')

In [ ]:
## merging aggregations together
#first left join all different newagg together. 
agg_cont_cat=pd.merge(agg_table1, agg_table2, how='left')

In [ ]:
list(agg_cont_cat)

In [ ]:
agg_cont_cat['pao2fio2ratio'].value_counts() 

In [ ]:
agg_cont_cat['any_vasoactive'].value_counts() 

In [ ]:
agg_cont_cat.head()

In [ ]:
pd.DataFrame(agg_cont_cat).to_csv(Path(
    wd+'/data/processed/merged/%s_newaggdf2_std.csv' %(date)),index=False)

In [ ]:
#YUS, got this reprocessed using a few of the top variables  4-16-19

In [ ]:
# big_categorical["icustay_id"].isin(icustay_id)

In [ ]:
#big_noCat['value']=big_noCat['value'].astype(float, copy=False)
#big_noCat.groupby(['icustay_id','label'], as_index=False)['value'].agg(dict1) #

In [ ]:
# test= big_noCat.groupby(['icustay_id'], as_index=False)['value'].agg(dict1)
# table = pd.pivot_table(big_noCat, values='value', columns='label', index=['icustay_id'],aggfunc=max, dropna=False)
# table

# pd.pivot_table(df,index=["Manager","Status"],columns=["Product"],values=["Quantity","Price"],
#                aggfunc={"Quantity":len,"Price":np.sum},fill_value=0)

In [ ]:
# #table = pivot_table(df, values=['D', 'E'], index=['A', 'C'],aggfunc={})
# ##don't know why this doesn't work. 
# table = pd.pivot_table(big_noCat, values='value',columns='label', index='icustay_id',aggfunc=dict1, dropna=False)
# table


In [ ]:
# big_agg= big_df.loc[big_df.loc[:,'label'].isin(non_category),:].groupby(['icustay_id','label'], as_index=False)['value'].apply(lambda x: float(x))

    #.agg({str(x):['max'] for x in hi_value})
#big_agg_count= big_agg.reset_index().pivot(index='icustay_id',columns='label', values=0)#, levels='icustay_id')

In [ ]:
# hi_df= big_df.loc[big_df.loc[:,'label'].isin(hi_value),:]
# hi_df['value']=hi_df['value'].astype(float, copy=False)
# hi_df.groupby(['icustay_id','label'], as_index=False)['value'].agg('max') #this works

In [ ]:
# %whos

# del big_agg
# del hi_df
# del final_pt_df2